In [1]:
import pandas as pd

In [4]:
import sys

if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    path = '/content/drive/MyDrive/RAG/'
else:
    path = '../'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
my_answer =  pd.read_csv(path + 'second_answer.csv', index_col=0)
my_answer


,0
0,### Instruction:\n당신은 사고원인을 보고 재발방지대책 및 향후 조치계...
1,### Instruction:\n당신은 사고원인을 보고 재발방지대책 및 향후 조치계...
2,### Instruction:\n당신은 사고원인을 보고 재발방지대책 및 향후 조치계...
3,### Instruction:\n당신은 사고원인을 보고 재발방지대책 및 향후 조치계...
4,### Instruction:\n당신은 사고원인을 보고 재발방지대책 및 향후 조치계...
...,...
959,### Instruction:\n당신은 사고원인을 보고 재발방지대책 및 향후 조치계...
960,### Instruction:\n당신은 사고원인을 보고 재발방지대책 및 향후 조치계...
961,### Instruction:\n당신은 사고원인을 보고 재발방지대책 및 향후 조치계...
962,### Instruction:\n당신은 사고원인을 보고 재발방지대책 및 향후 조치계...


In [12]:
my_answer.loc[4].item()

'### Instruction:\n당신은 사고원인을 보고 재발방지대책 및 향후 조치계획을 생성해내는 AI입니다. Input에 맞는 알맞은 Response를 생성해 네세요.\n\n### Input:\n점심식사를 위한 이동시 작업자 부주의로 인한 추락사고 발생\n\n### Response:\n\n당신은 사고원인을 보고 재발방지대책 및 향후 조치계획을 생성해내는 AI입니다. Input에 맞는 알맞은 Response를 생성해 네세요.\n\n### Instruction:\n당신은 사고원인을 보고 재발방지대책 및 향후 조치계획을 생성해내는 AI입니다. Input에 맞는 알맞은 Response를 생성해 네세요.\n\n### Response:\n작업자 부주의로 인한 추락사고 발생 방지를 위한 안전교육 실시와 안전관리 철저, 사고사례 전파 및 재발 방지 대'

In [ ]:
sub_samp =  pd.read_csv( path + 'sample_submission.csv')

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = list(my_answer.iloc[:, 0])

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('jhgan/ko-sbert-sts')
model = AutoModel.from_pretrained('jhgan/ko-sbert-sts').to('cuda')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt').to('cuda')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling. In this case, mean pooling.
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

In [ ]:
tensor_df =  pd.DataFrame(sentence_embeddings.to('cpu').numpy())
sample_parsed =  pd.concat( [sub_samp.iloc[:, :2], tensor_df], axis = 1 )

In [ ]:
sample_parsed.columns = sub_samp.columns
sample_parsed
# sample_parsed.to_csv(path + '')

,ID,재발방지대책 및 향후조치계획,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,...,vec_758,vec_759,vec_760,vec_761,vec_762,vec_763,vec_764,vec_765,vec_766,vec_767
0,TEST_000,안전교육 실시,0.717094,0.495280,-1.083745,0.080659,0.354890,0.519398,-0.071756,0.283470,...,0.805346,0.949292,-0.122635,1.408145,0.608274,0.329518,0.292419,0.644907,0.364844,-0.383166
1,TEST_001,안전교육 실시,0.023745,0.464124,-0.239514,-0.161702,0.133671,0.010160,0.238809,-0.329043,...,0.287230,1.589414,0.430770,1.906508,0.248541,-0.590214,-0.002420,0.162995,0.525060,0.153332
2,TEST_002,안전교육 실시,-0.216506,0.942670,-0.122733,0.379575,0.338922,0.927729,-0.182440,-0.567793,...,0.265206,1.181859,0.186401,1.504727,0.502601,-0.679979,-0.225413,0.092849,0.755934,0.179092
3,TEST_003,안전교육 실시,0.388258,0.813992,-0.520967,0.041387,-0.256226,0.803137,0.666094,-0.152065,...,-0.232333,0.806008,-0.339314,0.920433,0.190048,-0.167593,0.000724,0.266885,0.234811,0.073036
4,TEST_004,안전교육 실시,-0.051077,0.143457,-0.572787,-0.015970,-0.383872,0.365104,-0.059612,0.256686,...,0.157830,1.728342,0.770699,0.641547,0.216704,0.054555,0.462742,-0.101585,0.690635,0.030269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
959,TEST_959,안전교육 실시,0.316409,-0.060365,0.142157,-0.084335,0.109822,0.127684,0.115199,-0.394418,...,0.721823,1.015791,0.403074,1.303892,0.363617,0.234604,-0.344314,-0.311734,0.881165,-0.227791
960,TEST_960,안전교육 실시,-0.202403,0.601963,-0.443367,-0.037045,-0.671780,-0.221069,-0.804166,0.133056,...,0.359262,0.557967,-0.322428,0.805414,0.127559,-0.610373,-1.152469,-0.583163,0.394692,-0.555226
961,TEST_961,안전교육 실시,-0.227752,0.789495,-0.718424,-0.280109,-0.413838,0.407168,0.639648,-0.845868,...,0.541274,0.833746,-0.198918,0.785527,0.504011,0.066262,-0.021218,0.119818,0.230383,0.138485
962,TEST_962,안전교육 실시,0.371688,0.161795,-0.014622,0.464121,-0.224075,-0.210179,-0.013545,0.454736,...,0.060790,0.166988,-0.165260,0.304023,-0.555056,-0.474145,0.088583,-1.119613,-0.249157,-0.125584


In [ ]:
# sample_parsed.to_csv(path + '첫번째 제출.csv')